<a href="https://colab.research.google.com/github/fjadidi2001/Image_Inpaint/blob/main/Image_inpaint_New_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fsspec==2024.10.0
!pip install s3fs==2024.10.0
!pip install --upgrade fsspec gcsfs
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
!pip list

Package                            Version
---------------------------------- -------------------
absl-py                            1.4.0
accelerate                         1.1.1
aiobotocore                        2.15.2
aiohappyeyeballs                   2.4.3
aiohttp                            3.11.2
aioitertools                       0.12.0
aiosignal                          1.3.1
alabaster                          1.0.0
albucore                           0.0.19
albumentations                     1.4.20
altair                             4.2.2
annotated-types                    0.7.0
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.5.1
arviz                              0.20.0
astropy                            6.1.6
astropy-iers-data                  0.2024.11.18.0.35.2
astunparse                         1.6.3
async-timeout                      4.0.3
atpublic           

In [3]:
!pip uninstall gcsfs s3fs fsspec -y
!pip install gcsfs s3fs fsspec

Found existing installation: gcsfs 2024.10.0
Uninstalling gcsfs-2024.10.0:
  Successfully uninstalled gcsfs-2024.10.0
Found existing installation: s3fs 2024.10.0
Uninstalling s3fs-2024.10.0:
  Successfully uninstalled s3fs-2024.10.0
Found existing installation: fsspec 2024.10.0
Uninstalling fsspec-2024.10.0:
  Successfully uninstalled fsspec-2024.10.0
  Using cached s3fs-2024.10.0-py3-none-any.whl.metadata (1.7 kB)
Using cached s3fs-2024.10.0-py3-none-any.whl (29 kB)


In [4]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 54.4 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [3]:
!pip install datasets
from datasets import load_dataset

ds = load_dataset("saitsharipov/CelebA-HQ")


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import itertools

In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Model
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random
import gc
from tensorflow.keras.optimizers import Adam
import pandas as pd
import seaborn as sns
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras import mixed_precision
from datasets import load_dataset
from google.colab import drive
import itertools
# Mount Google Drive
drive.mount('/content/drive')

# # Enable memory growth for GPU and mixed precision training
# physical_devices = tf.config.list_physical_devices('GPU')
# if physical_devices:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)
# mixed_precision.set_global_policy('mixed_float16')

# Configuration
IMG_SIZE = 128  # Reduced from 256 due to memory constraints
BATCH_SIZE = 4
BUFFER_SIZE = 1000
CHECKPOINT_DIR = '/content/drive/My Drive/Inpaint'

def load_and_analyze_dataset():
    """Load images with memory efficiency"""
    ds = load_dataset("saitsharipov/CelebA-HQ")

    # Instead of loading all images at once, use a generator
    def image_generator():
        for img in ds['train']['image']:
            # Convert and resize to reduce memory
            yield np.array(img)

    # Limit the number of images
    image_paths = list(itertools.islice(image_generator(), 5000))

    print(f"Total images found: {len(image_paths)}")
    return image_paths

def create_mask(height, width):
    """Create random masks for inpainting"""
    mask = np.ones((height, width, 1))
    num_masks = np.random.randint(1, 4)  # Random number of masks
    for _ in range(num_masks):
        y1, x1 = np.random.randint(0, height - height//4), np.random.randint(0, width - width//4)
        h, w = np.random.randint(height//8, height//4), np.random.randint(width//8, width//4)
        mask[y1:y1+h, x1:x1+w] = 0
    return mask

def preprocess_image(img):
    """Load and preprocess images"""
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = tf.cast(img, tf.float32) / 255.0

    mask = create_mask(IMG_SIZE, IMG_SIZE)
    masked_img = img * mask

    return masked_img, mask, img

def augment(masked_img, mask, img):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
    masked_img = tf.image.random_flip_left_right(masked_img)
    mask = tf.image.random_flip_left_right(mask)
    return masked_img, mask, img

def create_dataset(image_paths):
    """Create TensorFlow dataset"""
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    return dataset

def unet_model():
    """Simplified U-Net with reduced complexity"""
    inputs = layers.Input((IMG_SIZE, IMG_SIZE, 3))

    # Reduce number of filters and layers
    conv1 = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.BatchNormalization()(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(64, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.BatchNormalization()(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Simplified bridge
    conv3 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.BatchNormalization()(conv3)

    # Reduced decoder complexity
    up1 = layers.UpSampling2D(size=(2, 2))(conv3)
    up1 = layers.concatenate([conv2, up1], axis=-1)
    conv4 = layers.Conv2D(64, 3, activation='relu', padding='same')(up1)
    conv4 = layers.BatchNormalization()(conv4)

    up2 = layers.UpSampling2D(size=(2, 2))(conv4)
    up2 = layers.concatenate([conv1, up2], axis=-1)
    conv5 = layers.Conv2D(32, 3, activation='relu', padding='same')(up2)
    conv5 = layers.BatchNormalization()(conv5)

    outputs = layers.Conv2D(3, 1, activation='sigmoid')(conv5)

    return Model(inputs=inputs, outputs=outputs)

def hint_model():
    """Create HINT model with transformer architecture"""
    inputs = layers.Input((IMG_SIZE, IMG_SIZE, 3))
    mask_input = layers.Input((IMG_SIZE, IMG_SIZE, 1))

    # Encode input with mask awareness
    x = layers.concatenate([inputs, mask_input])
    x = layers.Conv2D(64, 3, padding='same')(x)

    # Transformer blocks
    for _ in range(3):
        # Multi-head attention
        attention = layers.MultiHeadAttention(num_heads=4, key_dim=16)(x, x)
        x = layers.Add()([attention, x])
        x = layers.LayerNormalization()(x)

        # FFN
        ffn = layers.Dense(128, activation='relu')(x)
        ffn = layers.Dense(64)(ffn)
        x = layers.Add()([ffn, x])
        x = layers.LayerNormalization()(x)

    outputs = layers.Conv2D(3, 1, activation='sigmoid')(x)

    return Model(inputs=[inputs, mask_input], outputs=outputs)

def combined_model(unet, hint):
    """Combine U-Net and HINT models"""
    inputs = layers.Input((IMG_SIZE, IMG_SIZE, 3))
    mask_input = layers.Input((IMG_SIZE, IMG_SIZE, 1))

    unet_output = unet(inputs)
    hint_output = hint([inputs, mask_input])

    combined = layers.Average()([unet_output, hint_output])

    return Model(inputs=[inputs, mask_input], outputs=combined)

def perceptual_loss(y_true, y_pred):
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    loss_model = tf.keras.Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    return tf.reduce_mean(tf.square(loss_model(y_true) - loss_model(y_pred)))

def combined_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred)) + 0.1 * perceptual_loss(y_true, y_pred)

def evaluate_models(models, test_dataset):
    """Evaluate models using various metrics"""
    metrics = {
        'PSNR': [],
        'SSIM': [],
        'L1_Loss': []
    }

    for model_name, model in models.items():
        psnr_values = []
        ssim_values = []
        l1_values = []

        for masked_imgs, masks, original_imgs in test_dataset:
            if isinstance(model.input, list):
                predictions = model([masked_imgs, masks])
            else:
                predictions = model(masked_imgs)

            # Calculate metrics
            psnr = tf.image.psnr(original_imgs, predictions, max_val=1.0)
            ssim = tf.image.ssim(original_imgs, predictions, max_val=1.0)
            l1 = tf.reduce_mean(tf.abs(original_imgs - predictions))

            psnr_values.extend(psnr.numpy())
            ssim_values.extend(ssim.numpy())
            l1_values.append(l1.numpy())

        metrics['PSNR'].append(np.mean(psnr_values))
        metrics['SSIM'].append(np.mean(ssim_values))
        metrics['L1_Loss'].append(np.mean(l1_values))

    # Visualize metrics
    df = pd.DataFrame(metrics, index=list(models.keys()))
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df)
    plt.title('Model Comparison')
    plt.xticks(rotation=45)
    plt.show()

    return df

def visualize_inpainting(model, test_dataset, num_samples=5):
    for masked_imgs, masks, original_imgs in test_dataset.take(num_samples):
        if isinstance(model.input, list):
            predictions = model([masked_imgs, masks])
        else:
            predictions = model(masked_imgs)

        plt.figure(figsize=(15, 10))
        for i in range(num_samples):
            plt.subplot(num_samples, 3, i*3 + 1)
            plt.imshow(masked_imgs[i])
            plt.title('Masked Image')
            plt.axis('off')

            plt.subplot(num_samples, 3, i*3 + 2)
            plt.imshow(predictions[i])
            plt.title('Inpainted Image')
            plt.axis('off')

            plt.subplot(num_samples, 3, i*3 + 3)
            plt.imshow(original_imgs[i])
            plt.title('Original Image')
            plt.axis('off')
        plt.show()

def main():
    # GPU Memory Management
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        try:
            # Enable memory growth
            tf.config.experimental.set_memory_growth(physical_devices[0], True)

            # Limit GPU memory
            tf.config.experimental.set_virtual_device_configuration(
                physical_devices[0],
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]  # Limit to 4GB
            )
        except RuntimeError as e:
            print(e)

    # Reduce batch size and image resolution
    global IMG_SIZE, BATCH_SIZE
    IMG_SIZE = 128  # Reduced from 256
    BATCH_SIZE = 4  # Reduced batch size

    # 1. Load dataset and perform EDA
    image_paths = load_and_analyze_dataset()

    # 2. Split dataset
    train_paths, test_paths = train_test_split(image_paths, test_size=0.2, random_state=42)

    # 3. Create datasets
    train_dataset = create_dataset(train_paths).prefetch(tf.data.AUTOTUNE)
    test_dataset = create_dataset(test_paths).prefetch(tf.data.AUTOTUNE)

    # 4. Create models with reduced complexity
    unet = unet_model()
    hint = hint_model()
    combined = combined_model(unet, hint)

    # 5. Compile models
    optimizer = Adam(learning_rate=0.001)
    loss = combined_loss

    unet.compile(optimizer=optimizer, loss=loss)
    hint.compile(optimizer=optimizer, loss=loss)
    combined.compile(optimizer=optimizer, loss=loss)

    # Memory-efficient callbacks
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=0.00001
    )
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    # 7. Train models with memory management
    def train_with_memory_management(model, train_dataset, test_dataset):
        # Clear session before training
        tf.keras.backend.clear_session()

        # Train with limited epochs
        history = model.fit(
            train_dataset,
            validation_data=test_dataset,
            epochs=10,  # Limit epochs
            callbacks=[reduce_lr, early_stopping],
            verbose=1
        )

        # Explicitly clear memory
        del model
        gc.collect()

        return history

    # Train individual models
    unet_history = train_with_memory_management(unet, train_dataset, test_dataset)
    hint_history = train_with_memory_management(hint, train_dataset, test_dataset)

    # Train combined model
    combined_history = train_with_memory_management(combined, train_dataset, test_dataset)

    # 8. Evaluate models
    models = {
        'U-Net': unet,
        'HINT': hint,
        'Combined': combined
    }

    results = evaluate_models(models, test_dataset)
    print("\nEvaluation Results:")
    print(results)

    # 9. Visualize inpainting results
    visualize_inpainting(combined, test_dataset)

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ValueError: Cannot set memory growth on device when virtual devices configured